


# Artificial Neural Network (Ann) from Scratch

<font size= "4" color="black"> In this project Artificial Neural Network (ANN) was writeen from scratch.</font>










# Table of Content


    
 #### [1. Importing Libraries](#1) 

 #### [2. Activation Fucntions](#2)
    




<a id ="1"> </a>
# [Importing Libraries](#1)

<font size="4"> Library used in this project.
    

 <h5 style="color:red">1: Numpy  </h4>
 <h5 style="color:red">2: Math  </h4>
    
   
 </font>

In [1]:
#importing libraries

import numpy as np
#import random

import math

<a id ="2"> </a>



# [Activation Functions And Their Derivatives](#2)

<ul style="color:blue ">
    <li> <p style="font-size:16px">Relu </p> </li>
    <li> <p style="font-size:16px">Sigmoid </p> </li>
    <li> <p style="font-size:16px">Tanh </p> </li>
    <li> <p style="font-size:16px">Softmax </p> </li>

</ul>
        


In [2]:
#activation function
def relu(z):
    return np.maximum(z,0)

def derivative_of_relu(z):
    return np.array(z>0,dtype='float')

def sigmoid(z):
    return 1/(1+np.exp(-z))

def derivative_of_sigmoid(z):
    sg= sigmoid(z)
    return sg*(1-sg)

def tanh(z):
    return np.tanh(z)

def derivative_of_tanh(z):
    return 1-np.power(tanh(z),2)

def softmax(z):
    return np.exp(z)/np.sum(np.exp(z),axis=0)

<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Roboto&display=swap" rel="stylesheet">


# Layer Class


<p ><font face="arial" size="3">Perform Calculation for one Layer.</font></p>

### Methods 
<ul style="color:blue ">
    <li> <p style="font-size:16px">forward_propagation (input): <span style="color:black"> runs forward propagation for a single      layer </span> </p> </li>
    
    <li> <p style="font-size:16px">forward_propagation (input): <span style="color:black"> runs forward propagation for a single      layer </span> </p> </li>


</ul>


In [3]:
class Layer:
    def __init__(self,w_size,last_layer,activation):
        '''
            input:
                w_size: a list [no of input node, no of output node]
                last_layer: is this layer the last layer(True/False)
                activation: string (activation used in this layer)
                
        '''
        
        #parameters of this layer
        self.w= np.random.randn(w_size[1],w_size[0])
        self.dw=[]
        
        self.b= np.zeros((w_size[1],1))
        self.db=[]
        
        self.z=[]
        self.dz=[]
        
        self.last_layer= last_layer # is this the last layer
        
        self.x=[]  #input of this layer
        self.A= [] #output of this layer
        
        #activation used in this layer
        self.activation=activation




    def forward_propagation(self,A):
        '''
            input:
                A: input of this layer


            output:
                self.A: The output of this layer( activation(self.z))
        '''
        self.x=A
        self.z= np.array(self.w.dot(A)+ self.b)
        self.A= np.array(eval(self.activation+'(self.z)'))

        return self.A


    def back_propagation(self,y,w_next,dz_next,batch_size):
        '''
            input:
                
                y:an array of actual output of shape (no of class,no of samples)
                w_next: w parameter of the next layer(is not used in the last layer)
                dz_next: dz parameter of the next layer(is not used in the last layer)
                batch_size : batch size

            output:
                dw,dz : an array , value of dw and dz of this layer
        '''
        # for the last layer
        if self.last_layer:
            self.dz= np.array(self.A-y)
            #in the last layer, no of samples may not be the same as batch size
            batch_size= y.shape[1]
            
        # if not last layer
        else:
            #self.dz= self.np.array(w_next.T.dot(dz_next)) * derivative_of_sigmoid(self.z)
            self.dz= np.array(w_next.T.dot(dz_next)) * eval('derivative_of_'+self.activation+'(self.z)')


        #self.dw= (1/batch_size)*np.array(self.dz.dot(self.input.T))
        self.dw= (1/batch_size)*np.array(self.dz.dot(self.x.T))
        self.db= (1/batch_size)*np.sum(self.dz,axis=1,keepdims=True)

        return np.array(self.w),np.array(self.dz)

    def calculate_cost(self,y,_type):
        '''
            input:
                batch_size: batch size
                y: actual value of the output
                _type: type of cost function (binary classification / multiclassification)

            output: return the cost value
            Note: only the last layer calls this method
        '''
        #for binary classification
        if _type=='binary_classification':
            return np.sum((y*np.log(self.A))+(1-y)*np.log(1-self.A))*(-1/y.shape[1]) 

        #for multiclassification problem
        if _type=='multi_classification':
            return np.sum(y*np.log(self.A))/(-1/y.shape[1]) 

    def update_parameters(self,lr):
        '''
            lr: learning rate
            Note: Updates the parameter 
        '''
        self.w = self.w- (lr*self.dw)
        self.b= self.b- (lr*self.db)

In [43]:
class Ann:
    def __init__(self,neurons,activation,_type='binary_classification'):
        '''
            input:
                neurons: a list of integer , contains the number of neurons in each layer eg(10,12,10)
                activation: a list containing the name of activations for each layer
                _type: a string, binary classification or multi classification
        '''
        
        self.neurons= neurons
        self.activation= activation
       
        #contains all the layer objects
        self.layers=[]
        self._type= _type
        
        
        
    def train(self,_input,y_acc,batch_size,test_input=np.zeros((1,4)),test_label=np.zeros((1,4)),lr=0.1,iterations=1,show_acc=[True,True]):
            '''
                input:
                    _input: a numpy array, contains the input features , shape(input_features,sample no)
                    y_acc : an array of actual output, shape(no of class, sample no)
                    test_input: an array , contains the test input features , shape(input_features,test sample no)
                    test_label: an array , actual output of the test cases , shape(no of class,no of test sample)
                    
                    batch_size: an int, batch size for training
                    lr : a float , learning rate
                    iterations: an integer value, training iterations
                    show_acc: a list of bolean value ,show accuracy of [train,test] after each iteration
            '''

            #if there is no layer then create layer objects at first

            if len(self.layers)==0:
                #first value of the self.neuron will be the row of input array.
                self.neurons.insert(0,_input.shape[0])
                
                #creating layer objects
                for n in range(1,len(self.neurons)):
                    self.layers.append(Layer([self.neurons[n-1],self.neurons[n]],n== len(self.neurons)-1,self.activation[n-1]))

            #stores the shape of the original input
            _input_shape= [_input.shape[0],_input.shape[1]]
            
            #converting the _input and outputs to mini batches
            _input = [_input[:,j*batch_size:(j*batch_size)+batch_size] for j in range(math.ceil(_input.shape[1]/batch_size))]      
            y_acc= [y_acc[:,j*batch_size:(j*batch_size)+batch_size] for j in range(math.ceil(y_acc.shape[1]/batch_size))]
            
            
            #start the training 
            for i in range(iterations):
                cost =0 #contains the cost value of a whole iteration
                #for each batch
                for batch_x,batch_y in zip(_input,y_acc):
                    #input of the first layer
                    A = batch_x

                    #forward propagation for all layers
                    for layer in self.layers:
                        A= layer.forward_propagation(A)
                        
                    #calculate and add cost
                    cost += self.layers[-1].calculate_cost(batch_y,self._type)

                    #back propagation for all layers
                    w=dz=0
                    for layer in self.layers[::-1]:
                        w,dz= layer.back_propagation(batch_y,w,dz,batch_size)
                        #update parameters
                        layer.update_parameters(lr)
                
                # after each iteration
                #print train accuracy
                if show_acc[0]:
                    print('iter:{} - train_cost:{} - train_acc:{} -'.format(i,round(cost,2),self.predict(_input,y_acc,batch_size,True)[0]),
                        end=' ')
                
                #print test accuracy
                if show_acc[1]:
                    print('test_acc:{}'.format(self.predict(test_input,test_label,batch_size,False)[0]),end='')
                print() #draw a new line



    def predict(self,x,y,batch_size,train=False):

        '''
            input:
                x: an array of input features
                y_acc: an array of actual outputs
                batch_size: batch size
                train: is this training data or testing data (True for training data)

            output:
                return a list , [accuracy,predicted output]
        '''
        
        
        #divide only the test data into mini batches, train data are divided into batches in the train function
        if train==False:
            x = [x[:,j*batch_size:(j*batch_size)+batch_size] for j in range(math.ceil(x.shape[1]/batch_size))]
            y= [y[:,j*batch_size:(j*batch_size)+batch_size] for j in range(math.ceil(y.shape[1]/batch_size))]
        #contains the accuracy
        acc=[]
        #contains the predicted output
        y_pred=[]
        predicted_output= np.zeros((1,1))
        #counts how many samples are there
        m=0 
        #run forward propagation
        for batch_x ,batch_y in zip(x,y):
            a= batch_x
            for l in range(len(self.layers)):
                a= self.layers[l].forward_propagation(a)
            
            if np.sum(predicted_output)==0:
                predicted_output=a
                
            else:
                predicted_output= np.column_stack((predicted_output, a))
            
            if self._type== 'multi_classification':
                a= np.argmax(a,0)
                y_pred= np.argmax(batch_y,0)
                m+= a.shape[0]
                
            if self._type == "binary_classification":
                a=np.array(a>.5,dtype='float')
                y_pred= batch_y
                m+= a.shape[1]
                

            #m += a.shape[0]  #add the sample number in each batch to calculate avg accuracy
            acc.append(np.sum(np.array(a==y_pred,dtype='float')))

    
        return [round(100*sum(acc)/m,2),predicted_output]
       

In [57]:
# x= [[1,2,4],[5,6,7]]
# x= np.array(x).reshape(2,3)

# y=[[1,0,1],[0,1,0],[0,0,0],[0,0,0]]

# y= np.array(y).reshape(4,3)

# a= Ann([4],['softmax'],'multi_classification')
# a.train(x,y,2,x,y,.1,1,[True,True])